In [144]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/Continuous MPS/MatrixProductBP.jl`


In [145]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [146]:
seed = 4
rng = MersenneTwister(seed)

T = 4
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
N = 5
A = [0 1 1 1 1; 
     1 0 0 0 0; 
     1 0 0 0 0; 
     1 0 0 0 0; 
     1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

5×5 Matrix{Int64}:
 0  1  1  1  1
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0

In [147]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=2; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

Running MPBP: iter 2    Time: 0:00:00

In [148]:
# for (k,ed) in enumerate(edges(bp.g))
#     display(ed)
#     display(bp.μ[idx(ed)][3])
# end

In [149]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 1, 136578413690896)), 0, 1, false, false, 1.73753727410967e9, 1.73753727410967e9, 1.73753727410967e9)), [[[2.0, 1.9996646498695334, 1.9548049198724362, 1.9544998820940054, 1.9136950302803462], [2.0, 1.880797077977883, 1.8805416772783212, 1.8463767690747763, 1.8461444540853782], [2.0, 1.880797077977883, 1.8805416772783212, 1.8463767690747763, 1.8461444540853782], [2.0, 1.8807970779778824, 1.88054167727832, 1.8463767690747765, 1.8461444540853789], [2.0, 1.88079707797

In [150]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc=TruncThresh(0.0))
;

BoundsError: BoundsError: attempt to access 1×4×201×2 OffsetArray(::Array{ComplexF64, 4}, 1:1, 1:4, -100:100, 1:2) with eltype ComplexF64 with indices 1:1×1:4×-100:100×1:2 at index [OffsetArrays.IdOffsetRange(values=1:1, indices=1:1), OffsetArrays.IdOffsetRange(values=1:4, indices=1:4), 0]

In [151]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [152]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.0

(1, 1)

In [153]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.0

(Indexed Edge 1 => 2 with index 1, 1)

In [154]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1
marg = [0.10761390792083407, 0.892386092079166]
marg_fourier = [0.10761390792083407, 0.892386092079166]


In [155]:
node = 1
time = 3

edd = outedges(bp.g, node)
for ed in edd
    display(ed)
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

Indexed Edge 1 => 2 with index 1

2-element Vector{Float64}:
 0.10761390792083407
 0.892386092079166

2-element Vector{Float64}:
 0.10761390792083407
 0.892386092079166

Indexed Edge 1 => 3 with index 2

2-element Vector{Float64}:
 0.10761390792083407
 0.892386092079166

...

2-element Vector{Float64}:
 0.10761390792083177
 0.8923860920791682

2-element Vector{Float64}:
 0.10761390792083177
 0.8923860920791682

Indexed Edge 1 => 5 with index 4

2-element Vector{Float64}:
 0.10761390792083152
 0.8923860920791684

2-element Vector{Float64}:
 0.10761390792083152
 0.8923860920791684

In [156]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 0.0
 1.0

2-element Vector{Float64}:
 0.0
 1.0

"time = 2"

...

2-element Vector{Float64}:
 0.5
 0.5

2-element Vector{Float64}:
 0.5
 0.5

"time = 5"

2-element Vector{Float64}:
 0.5000000000000001
 0.4999999999999999

2-element Vector{Float64}:
 0.5000000000000001
 0.4999999999999999